<a href="https://colab.research.google.com/github/Gram012/8581_BigData/blob/main/Copy_of_8581_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

-Make a copy of this notebook: File → "Save a copy in Drive" (Optional if doing solo stuff)

-Connect to GPU runtime: Runtime → "Change runtime type" → "T4 GPU"

  I'm not totally sure what Colab's limits on GPU usage are, but it might be a good idea to have only one member of your group connect to a GPU runtime, while the rest stay on CPU. Most of this notebook can be run without a GPU, and if one person runs out of GPU credits, another can swap in



-Mount Google Drive with code block below. You'll need to grant Google Drive some permissions for this to work

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


-Open up the file explorer in the left sidebar (using the folder icon) and locate the zip file in your Drive. In the cell below, replace the path with the path to your file, and run the cell to unzip it.

In [ ]:
# Fill in the path to your copy of the zip file
# Should take ~8 minutes :( The %%capture is there to suppress output
%%capture
!unzip drive/MyDrive/8581_Project/PHYS8581_data.zip

In [ ]:
# Fixes a bug that comes up sometimes
# If you get an error involving a "UTF-8 encoding", uncomment and run the below

# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

-Run the cell below to do a little organization and clean-up (not needed right now)

In [ ]:
# For convenience, move everything into the top-level, and remove a file
# that osx puts in zips
#!mv PHYS8581_data/* /content/
#!rm -r __MACOSX

Imports!

In [ ]:
import h5py
import glob
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
file_waveforms = '/content/PHYS8581_data/waveforms.hdf5'


with h5py.File(file_waveforms, 'r') as f:
    print(list(f.keys()))
    parameters = f['parameters']
    print("Parameters:", list(parameters.keys()))

['parameters', 'waveforms']
Parameters: ['a_1', 'a_2', 'dec', 'ifo_snrs', 'mass_1', 'mass_2', 'phase', 'phi_12', 'phi_jl', 'psi', 'ra', 'redshift', 'snr', 'theta_jn', 'tilt_1', 'tilt_2']


Contents of the README.txt:

The background data in this directory is real data from LIGO's third observing run, O3.


Each file name contains the GPS time of the start of the segment, as well as the duration of the segment in seconds.


The data is sampled at 2048 Hz.


Each file contains two data channels, "H1" and "L1", corresponding to data from the Hanford and Livingston interferometers.


The detectors are located on opposite sides of the United States, so the noise in the detectors should be uncorrelated.


This means that it's entirely valid to slice samples from these files non-coincidentally for training/testing: you can take windows from H1 and L1 that are at different times.


This is a good way to increase the effective size of your dataset.

The waveforms.hdf5 file contains 10,000 simulated binary neutron star (BNS) waveforms.
The specifics are as follows:
 - Waveform approximant: TaylorF2
 - Minimum frequency: 20 Hz
 - Reference frequency: 50 Hz
 - Sample rate: 2048 Hz
 - Waveform duration: 60 s
 - Highpass frequency: 20 Hz
 - Parameter distributions:
    - Mass 1: Triangular(1, 2.5, 2.5)
    - Mass 2: Uniform(1, Mass 1)
    - Spins and spin angles: 0
    - Sky location: Isotropic
    - Redshift: Uniform in volume between 0 and 0.01 (~43 Mpc)

The upper bound on redshift was chosen to roughly match the distance of GW170817, the one BNS signal that's been detected.


As a consequence, almost all of the signals in the dataset are relatively loud and should be detectable.


If this ends up being too simple of a problem, the waveform dataset can be regenerated using a higher maximum redshift.

Note that these signals have not been added into background, meaning that you can add the same signal into different background segments to get different training examples.

Starting with injecting waveforms